The goal of this notebook is to create a function that outputs the mean and standard deviation of a Weibull distribution, given its location and scale parameters.

In [114]:
from scipy.special import gamma, digamma
import numpy as np

In [35]:
def weibull_mu_and_sigma(alpha, beta):
    """Given the shape (alpha) and scale (beta) parameters of a Weibull distribution,
    outputs the corresponding mean and standard deviation as a 2-tuple.
    """
    mu = beta*gamma(1 + (1/alpha))
    sigma = (beta**2)*gamma(1 + (2/alpha) - (mu**2)/(beta**2))
    return mu, sigma

In [36]:
mu, sigma = weibull_mu_and_sigma(2,1)
print(f"mu = {mu}")
print(f"sigma = {sigma}")

mu = 0.8862269254527579
sigma = 0.9144246099120455


# Can we go in reverse using an optimizer?

In [115]:
from scipy import optimize

In [116]:
def fun(x, mu, sigma):
    """Function for finding alpha and beta given a mu and sigma.
    First entry is alpha, second is beta.
    """
    alpha = x[0]
    beta = x[1]
    return [beta*gamma(1 + (1/alpha)) - mu,
            (beta**2)*gamma(1 + (2/alpha) - (mu**2)/(beta**2)) - sigma]

In [46]:
def weibull_alpha_and_beta(mu, sigma):
    """Given the mean (mu) and standard deviation (sigma) parameters of a Weibull distribution,
    outputs the corresponding shape (alpha) and scale (beta) as a 2-tuple.
    """

    def fun(x, mu, sigma):
        """Function called for solving the system for alpha and beta given a mu and sigma.
        First entry is in x alpha, second is beta.
        """
        alpha = x[0]
        beta = x[1]
        return [beta*gamma(1 + (1/alpha)) - mu,
                (beta**2)*gamma(1 + (2/alpha) - (mu**2)/(beta**2)) - sigma]

    res = optimize.root(fun, [1.0, 1.0], args=(mu,sigma))

    if res.success == False:
        raise Exception("Solver for alpha and beta failed.")
    else:
        alpha, beta = res.x
        return alpha, beta

In [118]:
def fun(x, mu, sigma):
    """Function called for solving the system for alpha and beta given a mu and sigma.
    First entry is in x alpha, second is beta.
    """
    alpha = x[0]
    beta = x[1]

    f_val = [beta*gamma(1 + (1/alpha)) - mu,
            (beta**2)*gamma(1 + (2/alpha) - (mu**2)/(beta**2)) - sigma]

    df1_dalpha = -beta*gamma(1 + (1/alpha))*digamma(1 + (1/alpha))/(alpha**2)
    df1_dbeta = gamma(1 + (1/alpha))
    
    temp = 1 + (2/alpha) - ((mu**2)/(beta**2))
    df2_dalpha = -2*(beta**2)*gamma(temp)*digamma(temp)/(alpha**2)
    df2_dbeta = 2*gamma(temp)*( (mu**2)*digamma(temp) + (beta**2))/beta


    f_jac = np.array([[df1_dalpha, df1_dbeta],
                    [df2_dalpha, df2_dbeta]])

    return f_val, f_jac

In [152]:
mu, sigma = 10, 4

In [159]:
res = optimize.root(fun, [5, 10], args=(mu,sigma), jac=True)

In [160]:
res

    fjac: array([[-5.29395592e-23,  1.00000000e+00],
       [-1.00000000e+00, -5.29395592e-23]])
     fun: array([-10.,  -4.])
 message: 'The iteration is not making good progress, as measured by the \n  improvement from the last ten iterations.'
    nfev: 14
    njev: 2
     qtf: array([-4., 10.])
       r: array([2.96543897e+77, 4.47345670e+77, 1.14179815e+46])
  status: 5
 success: False
       x: array([-1.89820479e-03,  9.40872372e+00])

In [155]:
alpha, beta = res.x

In [156]:
r1, r2 = weibull_mu_and_sigma(alpha, beta)
print(r1)
print(r2)

4.7668687571496555
11.497243585972505


In [ ]:
alpha, beta = weibull_alpha_and_beta(1, 3)

In [93]:
alpha, beta = weibull_alpha_and_beta(0.9, 3)

Exception: Solver for alpha and beta failed.

In [94]:
r1, r2 = weibull_mu_and_sigma(alpha, beta)

In [95]:
print(r1)
print(r2)

0.9499999999999952
2.9999999999983578


In [98]:
mu, sigma = 0.9, 1

In [110]:
res = optimize.root(fun, [10.0, 5.0], args=(mu,sigma), options={"maxfev":100000, "eps":1e-8})

In [111]:
res

    fjac: array([[-0.02071979, -0.99978532],
       [ 0.99978532, -0.02071979]])
     fun: array([0.35918807, 1.88680893])
 message: 'The iteration is not making good progress, as measured by the \n  improvement from the last ten iterations.'
    nfev: 41
     qtf: array([-1.89384617,  0.32001669])
       r: array([-5.74294258e-05,  1.62556874e-01,  1.00432252e+00])
  status: 5
 success: False
       x: array([-783.04798655,    1.25825853])

In [107]:
help(res)

Help on OptimizeResult in module scipy.optimize.optimize object:

class OptimizeResult(builtins.dict)
 |  Represents the optimization result.
 |  
 |  Attributes
 |  ----------
 |  x : ndarray
 |      The solution of the optimization.
 |  success : bool
 |      Whether or not the optimizer exited successfully.
 |  status : int
 |      Termination status of the optimizer. Its value depends on the
 |      underlying solver. Refer to `message` for details.
 |  message : str
 |      Description of the cause of the termination.
 |  fun, jac, hess: ndarray
 |      Values of objective function, its Jacobian and its Hessian (if
 |      available). The Hessians may be approximations, see the documentation
 |      of the function in question.
 |  hess_inv : object
 |      Inverse of the objective function's Hessian; may be an approximation.
 |      Not available for all solvers. The type of this attribute may be
 |      either np.ndarray or scipy.sparse.linalg.LinearOperator.
 |  nfev, njev, nhe

In [32]:
res.success

True

In [22]:
res.x

array([5.37797259, 1.08457197])